<p style="page-break-after:always;"></p>

UW user id: `g66xu`

# Problem 1
For part (a), (b), and (d) I wrote a program to obtain the results. The code can be found at the end of this problem's writeup.

## a)
Given the partial key, the bias of the linear relationship is $0.00205$ over my inputs

## b)
The output of my program is as follows:

```
K5 candidate: 0x0004, observed bias: 0.026800
K5 candidate: 0x0009, observed bias: 0.026800
K5 candidate: 0x0304, observed bias: 0.026350
K5 candidate: 0x0d04, observed bias: 0.026000
K5 candidate: 0x0104, observed bias: 0.024650
```

Hence the most probable partial key is `0b0000` for bit 5-8 and `0b0100` or `0b1001` (equally probable) for bit 13-16

## c)
Finding the bias of the individual linear relationship involves computing the input sum and the output sum. Then we use table 4 to look up the expected bias. The results are as follows:

|linear relation|input sum|output sum|expected bias|
|:---|:---|:---|:---|
|$S_{11}: X_1 \oplus X_4 \approx Y_1$|9|8|-4/16|
|$S_{13}: X_1 \oplus X_4 \approx Y_1$|9|8|-4/16|
|$S_{21}: X_1 \oplus X_3 \approx Y_2$|10|4|-4/16|
|$S_{32}: X_1 \approx Y_1 \oplus Y_2 \oplus Y_3 \oplus Y_4 $|8|15|-6/16|

We can trace through the Heys Cipher write down the linear relationships of the intermediary state before arriving at a final linear approximation:

$$
\begin{aligned}
U_{1,1} &= P_1 \oplus K_{1,1} \\
U_{1,4} &= P_4 \oplus K_{1,4} \\
U_{1,9} &= P_9 \oplus K_{1,9} \\
U_{1,12} &= P_{12} \oplus K_{1,12} \\
V_{1,1} &\approx U_{1,1} \oplus U_{1,4}, \epsilon = -\frac{1}{4} \\
V_{1,9} &\approx U_{1,9} \oplus U_{1,12}, \epsilon = -\frac{1}{4} \\
U_{2,1} &= V_{1,1} \oplus K_{2,1} \\
U_{2,3} &= V_{1,9} \oplus K_{2,3} \\
V_{2,2} &\approx U_{2,1} \oplus U_{2,3}, \epsilon = -\frac{1}{4} \\
U_{3,5} &= V_{2,2} \oplus K_{3,5} \\
V_{3,5} \oplus V_{3,6} \oplus V_{3,7} \oplus V_{3,8} &\approx U_{3,5}, \epsilon = -\frac{3}{8}
\end{aligned}
$$

Therefore:

$$
\begin{aligned}
V_{3,5} \oplus V_{3,6} \oplus V_{3,7} \oplus V_{3,8} &\approx U_{3,5} \\
&\approx V_{2,2} \oplus K_{3,5} \\
&\approx U_{2,1} \oplus U_{2,3} \oplus K_{3,5} \\
&\approx (V_{1,1} \oplus K_{2,1}) \oplus (V_{1,9} \oplus K_{2,3}) \oplus K_{3,5} \\
&\approx U_{1,1} \oplus U_{1,4} \oplus K_{2,1} \oplus U_{1,9} \oplus U_{1,12} \oplus K_{3,5} \\
&\approx P_1 \oplus K_{1,1} \oplus P_4 \oplus K_{1,4} \oplus K_{2,1} \oplus P_9 \oplus K_{1,9} \oplus P_{12} \oplus K_{1,12} \oplus K_{3,5} \\
\end{aligned}
$$

Add to the approximation above the following relationship:

$$
\begin{aligned}
U_{4,2} &= V_{3,5} \oplus K_{4,2} \\
U_{4,6} &= V_{3,6} \oplus K_{4,6} \\
U_{4,10} &= V_{3,7} \oplus K_{4,10} \\
U_{4,14} &= V_{3,8} \oplus K_{4,14} \\
\end{aligned}
$$

Finally, move everything to onde side, We have the following approximation:

$$
P_1 \oplus P_4 \oplus P_9 \oplus P_{12} 
\oplus U_{4,2} \oplus U_{4,6} \oplus U_{4,10} \oplus U_{4,14}
\oplus \sum\text{bunch of key bits}
\approx 0
$$

Where the bunch of key bits include: $K_{1,1}, K_{1,4}, K_{1,9}, K_{1,12}, K_{2,1}, K_{3,5}, K_{4,2}, K_{4,6}, K_{4,10}, K_{4,14}$

The final bias can be approximated using the piling-up lemma:

$$
\epsilon = 2^3 \cdot \Pi_{i=1}^{4}\epsilon_i = 8 \cdot (-\frac{1}{4}) \cdot (-\frac{1}{4}) \cdot (-\frac{1}{4}) \cdot (-\frac{3}{8}) = \frac{3}{64}
$$

Also, a helpful diagram:

<img src="./static/a2q1c.png" width=500></img>

## d)
The output of my program is as follows:

```
K5 candidate: 0xd019, observed bias: 0.052650
K5 candidate: 0xd619, observed bias: 0.029800
K5 candidate: 0xc019, observed bias: 0.029150
K5 candidate: 0xd419, observed bias: 0.029150
K5 candidate: 0xd01a, observed bias: 0.029050
```

Notice that `0xd019` has particularly high bias compared to the second-ranked candidate, which makes it highly probably that `0xd019` is the most probable candidate for $K_5$

## e)
Sorry but I am out of ideas

## Appendix:
Source code (written in Rust, btw).

First, an implementation of the Heys' Cipher

```rust
//! Implementation of the Heys' Cipher
use std::error::Error;

type Result<T> = core::result::Result<T, Box<dyn Error>>;

const SBOX: [(u16, u16); 16] = [
    (0x0, 0xE),
    (0x1, 0x4),
    (0x2, 0xD),
    (0x3, 0x1),
    (0x4, 0x2),
    (0x5, 0xF),
    (0x6, 0xB),
    (0x7, 0x8),
    (0x8, 0x3),
    (0x9, 0xA),
    (0xA, 0x6),
    (0xB, 0xC),
    (0xC, 0x5),
    (0xD, 0x9),
    (0xE, 0x0),
    (0xF, 0x7),
];

const SBOX_INVERT: [(u16, u16); 16] = [
    (0xE, 0x0),
    (0x4, 0x1),
    (0xD, 0x2),
    (0x1, 0x3),
    (0x2, 0x4),
    (0xF, 0x5),
    (0xB, 0x6),
    (0x8, 0x7),
    (0x3, 0x8),
    (0xA, 0x9),
    (0x6, 0xA),
    (0xC, 0xB),
    (0x5, 0xC),
    (0x9, 0xD),
    (0x0, 0xE),
    (0x7, 0xF),
];

const PERMUTATION: [(u16, u16); 16] = [
    (0x8000, 0x8000),
    (0x4000, 0x0800),
    (0x2000, 0x0080),
    (0x1000, 0x0008),
    (0x0800, 0x4000),
    (0x0400, 0x0400),
    (0x0200, 0x0040),
    (0x0100, 0x0004),
    (0x0080, 0x2000),
    (0x0040, 0x0200),
    (0x0020, 0x0020),
    (0x0010, 0x0002),
    (0x0008, 0x1000),
    (0x0004, 0x0100),
    (0x0002, 0x0010),
    (0x0001, 0x0001),
];

const PERMUTATION_INVERT: [(u16, u16); 16] = [
    (0x8000, 0x8000),
    (0x0800, 0x4000),
    (0x0080, 0x2000),
    (0x0008, 0x1000),
    (0x4000, 0x0800),
    (0x0400, 0x0400),
    (0x0040, 0x0200),
    (0x0004, 0x0100),
    (0x2000, 0x0080),
    (0x0200, 0x0040),
    (0x0020, 0x0020),
    (0x0002, 0x0010),
    (0x1000, 0x0008),
    (0x0100, 0x0004),
    (0x0010, 0x0002),
    (0x0001, 0x0001),
];

fn lookup(kvpairs: &[(u16, u16)], val: u16) -> Result<u16> {
    for (from, to) in kvpairs {
        if *from == val {
            return Ok(*to);
        }
    }
    return Err("Lookup failed".into());
}

/// A 16-bit block, could be a plaintext, ciphertext, or some intermediary
/// state
#[derive(Debug, Eq, PartialEq)]
pub struct Block {
    val: u16,
}

impl Block {
    pub fn new(val: u16) -> Self {
        return Self { val };
    }

    /// Get block from some 16-character binary string
    pub fn from_binstr(binstr: &str) -> Result<Self> {
        let val = u16::from_str_radix(binstr, 2)?;
        return Ok(Self::new(val));
    }

    /// XOR with round key
    pub fn mix_key(&self, key: u16) -> Self {
        return Self::new(self.val ^ key);
    }

    /// Get the bit at the specified location following big-endianness and
    /// 1-based indexing
    pub fn get_bit_1base(&self, loc: u8) -> Result<u16> {
        if loc < 1 || loc > 16 {
            return Err("loc must be between 1 and 16".into());
        }
        let mask = 1u16 << (16 - loc);
        if self.val & mask == 0 {
            return Ok(0);
        }
        return Ok(1);
    }

    /// Substitute each 4-bit block according to the input SBOX, then put the
    /// substituted blocks back together
    fn substitute(&self, sbox: &[(u16, u16)]) -> Result<Self> {
        let b0 = self.val % 16;
        let b1 = (self.val >> 4) % 16;
        let b2 = (self.val >> 8) % 16;
        let b3 = (self.val >> 12) % 16;

        let b0 = lookup(&sbox, b0)?;
        let b1 = lookup(&sbox, b1)?;
        let b2 = lookup(&sbox, b2)?;
        let b3 = lookup(&sbox, b3)?;

        let b3 = b3 << 12;
        let b2 = b2 << 8;
        let b1 = b1 << 4;

        return Ok(Self::new(b3 + b2 + b1 + b0));
    }

    /// Permute each bit according to the input permutation
    fn permute(&self, permutation: &[(u16, u16)]) -> Result<Self> {
        let mut val: u16 = 0;

        for shift in 0..16 {
            let mask = 1u16 << shift;
            if (self.val & mask) != 0 {
                let mapped = lookup(permutation, mask)?;
                val += mapped;
            }
        }

        return Ok(Self::new(val));
    }
}

/// Heys' block cipher
pub struct HeysCipher {
    round_keys: [u16; 5],
}

impl HeysCipher {
    pub fn from_keys(new_keys: &[u16]) -> Result<Self> {
        if new_keys.len() != 5 {
            return Err("Cipher requires exactly 5 round keys".into());
        }
        let mut round_keys: [u16; 5] = [0; 5];
        for i in 0..5 {
            round_keys[i] = new_keys[i];
        }
        return Ok(Self { round_keys });
    }

    pub fn encrypt(&self, plaintext: &Block) -> Result<Block> {
        let u1 = plaintext.mix_key(self.round_keys[0]);
        let v1 = u1.substitute(&SBOX)?;
        let u2 = v1.permute(&PERMUTATION)?.mix_key(self.round_keys[1]);
        let v2 = u2.substitute(&SBOX)?;
        let u3 = v2.permute(&PERMUTATION)?.mix_key(self.round_keys[2]);
        let v3 = u3.substitute(&SBOX)?;
        let u4 = v3.permute(&PERMUTATION)?.mix_key(self.round_keys[3]);
        let v4 = u4.substitute(&SBOX)?;

        return Ok(v4.mix_key(self.round_keys[4]));
    }

    pub fn decrypt(&self, ciphertext: &Block) -> Result<Block> {
        let v4 = ciphertext.mix_key(self.round_keys[4]);
        let u4 = v4.substitute(&SBOX_INVERT)?;
        let v3 = u4
            .mix_key(self.round_keys[3])
            .permute(&PERMUTATION_INVERT)?;
        let u3 = v3.substitute(&SBOX_INVERT)?;
        let v2 = u3
            .mix_key(self.round_keys[2])
            .permute(&PERMUTATION_INVERT)?;
        let u2 = v2.substitute(&SBOX_INVERT)?;
        let v1 = u2
            .mix_key(self.round_keys[1])
            .permute(&PERMUTATION_INVERT)?;
        let u1 = v1.substitute(&SBOX_INVERT)?;
        let pt = u1.mix_key(self.round_keys[0]);

        return Ok(pt);
    }

    /// A linear approximation takes the form:
    ///
    /// (some plaintext bits) + (some U4 bits) = 0 (mod 2)
    /// where U4 is the intermediary state and the bits are specified using
    /// big-endian, 1-based indexing.
    ///
    /// For example:
    /// U[4,6] + U[4,8] + U[4,14] + U[4,16] + P[5] + P[7] + P[8] == 0
    /// (this is stated in Heys' paper in section 3.4)
    ///
    /// User is responsible for making sure the input bit locs are valid
    pub fn check_linear_approx(
        &self,
        pt: &Block,
        ct: &Block,
        pt_locs: &[u8],
        u4_locs: &[u8],
    ) -> u16 {
        let mut binsum: u16 = 0;
        let u4 = ct
            .mix_key(self.round_keys[4])
            .substitute(&SBOX_INVERT)
            .unwrap();
        binsum += pt_locs
            .iter()
            .map(|loc| pt.get_bit_1base(*loc).unwrap())
            .sum::<u16>();
        binsum += u4_locs
            .iter()
            .map(|loc| u4.get_bit_1base(*loc).unwrap())
            .sum::<u16>();

        return 1 - binsum % 2;
    }

    /// Compute the bias of the input linear relation over the inputs PT-CT
    /// pairs
    pub fn get_bias(
        &self,
        plaintexts: &[Block],
        ciphertexts: &[Block],
        pt_locs: &[u8],
        u4_locs: &[u8],
    ) -> f64 {
        let sum = plaintexts
            .iter()
            .zip(ciphertexts.iter())
            .map(|(pt, ct)| self.check_linear_approx(pt, ct, pt_locs, u4_locs))
            .sum::<u16>();
        let prob = (sum as f64) / (plaintexts.len() as f64);
        if prob > 0.5 {
            return prob - 0.5;
        }
        return 0.5 - prob;
    }
}

/// Given in the input PT-CT pairs and the relationship specified by P-locs
/// U4-locs, return all possible K5 values, ranked by bias
pub fn brute_force_k5(
    plaintexts: &[Block],
    ciphertexts: &[Block],
    pt_locs: &[u8],
    u4_locs: &[u8],
    verbose: bool,
) -> Vec<(f64, u16)> {
    let mut rankings: Vec<(f64, u16)> = (u16::MIN..=u16::MAX)
        .map(|round_key| {
            if verbose {
                println!("{round_key}");
            }
            let cipher = HeysCipher::from_keys(&[0, 0, 0, 0, round_key]).unwrap();
            let bias = cipher.get_bias(plaintexts, ciphertexts, pt_locs, u4_locs);
            return (bias, round_key);
        })
        .collect::<Vec<(f64, u16)>>();
    rankings.sort_by(|elem1, elem2| {
        let (bias1, _) = elem1;
        let (bias2, _) = elem2;
        return bias2.partial_cmp(bias1).unwrap(); // reverse sort
    });
    return rankings;
}
```

Source code for part (a)

```rust
use cryptanalysis::heys::{Block, HeysCipher};
use std::fs;

fn read_inputs(path: &str) -> Vec<Block> {
    return fs::read_to_string(path)
        .unwrap()
        .lines()
        .map(|line| Block::from_binstr(line).unwrap())
        .collect::<Vec<Block>>();
}

fn main() {
    let plaintexts = read_inputs("./inputs/a2q1plaintexts.txt");
    let ciphertexts = read_inputs("./inputs/a2q1ciphertexts.txt");
    let guess = HeysCipher::from_keys(&[0, 0, 0, 0, 0b0000011100000110]).unwrap();

    let bias = guess.get_bias(&plaintexts, &ciphertexts, &[5, 7, 8], &[6, 8, 14, 16]);
    println!("bias: {bias:.08}");
}
```

part (b)

```rust
use cryptanalysis::heys::{Block, HeysCipher};
use std::fs;

fn read_inputs(path: &str) -> Vec<Block> {
    return fs::read_to_string(path)
        .unwrap()
        .lines()
        .map(|line| Block::from_binstr(line).unwrap())
        .collect::<Vec<Block>>();
}

fn main() {
    let plaintexts = read_inputs("./inputs/a2q1plaintexts.txt");
    let ciphertexts = read_inputs("./inputs/a2q1ciphertexts.txt");

    let mut rankings: Vec<(f64, u16)> = vec![];
    for bits_5_to_8 in 0b0000u16..=0b1111 {
        for bits_13_to_16 in 0b0000u16..=0b1111 {
            let round_key = (bits_5_to_8 << 8) + bits_13_to_16;
            let guess = HeysCipher::from_keys(&[0, 0, 0, 0, round_key]).unwrap();
            let bias = guess.get_bias(&plaintexts, &ciphertexts, &[5, 7, 8], &[6, 8, 14, 16]);
            rankings.push((bias, round_key));
        }
    }

    rankings.sort_by(|elem1, elem2| {
        let (bias1, _) = elem1;
        let (bias2, _) = elem2;
        return bias2.partial_cmp(bias1).unwrap();
    });
    rankings.iter().take(5).for_each(|(bias, round_key)| {
        println!("K5 candidate: 0x{round_key:04x}, observed bias: {bias:.6}");
    });
}
```

part (d)

```rust
use cryptanalysis::heys::{self, Block};
use std::fs;

fn read_inputs(path: &str) -> Vec<Block> {
    return fs::read_to_string(path)
        .unwrap()
        .lines()
        .map(|line| Block::from_binstr(line).unwrap())
        .collect::<Vec<Block>>();
}

fn main() {
    let plaintexts = read_inputs("./inputs/a2q1plaintexts.txt");
    let ciphertexts = read_inputs("./inputs/a2q1ciphertexts.txt");
    let rankings = heys::brute_force_k5(
        &plaintexts,
        &ciphertexts,
        &[1, 4, 9, 12],
        &[2, 6, 10, 14],
        false,
    );

    rankings.iter().take(5).for_each(|(bias, round_key)| {
        println!("K5 candidate: 0x{round_key:04x}, observed bias: {bias:.6}");
    });
}
```

<p style="page-break-after:always;"></p>

UW user id: `g66xu`

# Problem 2

## a)
we claim without proof that $\overline{a} \oplus \overline{b} = a \oplus b$ and $\overline{a} \oplus b = \overline{a \oplus b}$. They can be trivially proved using a truth table.

Because the message expansion of the component function is a permutation expansion, it is a linear transformation. Therefore, the negation of the expansion is identical to the expansion of the negation:

$$
\text{Expansion}(\bar{m}) = \overline{\text{Expansion}(m)}
$$

Now we are ready to show that negating both the message and the key will not affect the output of the component function. This is true because the expanded message and the key are mixed before being sent to S-Box and the permutation, so negating both the message and the key will not change the input to the S-Box.

$$
\begin{aligned}
f_i(\overline{k_i}, \overline{m}) &= \text{Permutation}(\text{S-Box}(\text{Exp}(\overline{m}) \oplus \overline{k_i})) \\
&= \text{Permutation}(\text{S-Box}(\overline{\text{Exp}(m)} \oplus \overline{k_i})) \\
&= \text{Permutation}(\text{S-Box}(\text{Exp}(m) \oplus k_i)) \\
&= f_i(k_i, m)
\end{aligned}
$$

## b)
We begin by claiming that the key scheduling function is a linear transformation. This is true because the key scheduling function only performs permutation and bit shifts, both being linear transformation. Therefore, negating the master key will negate all round keys.

We will prove that the negation propagate across the entire Feistel network through induction. First, we will show that negation will propagate through a single round of the Feistel network:

$$
\begin{aligned}
F_i(\overline{k_i}, \overline{L_i} \Vert \overline{R_i}) &= \overline{R_i} \Vert (\overline{L_i} \oplus f_i(\overline{k_i}, \overline{R_i})) \\
&= \overline{R_i} \Vert (\overline{L_i} \oplus f_i(k_i, R_i)) \\
&= \overline{R_i} \Vert (\overline{L_i \oplus f_i(k_i, R_i)}) \\
&= \overline{R_i \Vert (L_i \oplus f_i(k_i, R_i))} \\
&= \overline{F_i(k_i, L_i \Vert R_i)}
\end{aligned}
$$

We can then show that the negation will also propagate through any two rounds of Feistel network:

$$
\begin{aligned}
F_{i+1}(\overline{k_{i+1}}, F_i(\overline{k_i}, \overline{L_i} \Vert \overline{R_i}))
&= F_{i+1}(\overline{k_{i+1}}, \overline{F_i(k_i, L_i \Vert R_i)}) \\
&= F_{i+1}(\overline{k_{i+1}}, \overline{L_{i+1}} \Vert \overline{R_{i+1}}) \\
&= \overline{F_{i+1}(k_{i+1}, L_{i+1} \Vert R_{i+1})} \\
&= \overline{F_{i+1}(k_{i+1}, F_i(k_i, L_i \Vert R_i)}
\end{aligned}
$$

By mathematical induction, we can conclude that **negation will propagate through arbitrary number of rounds of Feistel network**, hence it will propagate through the 16-round Feistel network in DES.

Last but not least, observe that both the initial permutation and the final permutation are both linear transformation, so negation will propagate through these two steps as well. Thus we have proved that negation of the key and the message will propagation through all steps of the DES construction. Therefore:

$$
\text{DES}(k, m) = \overline{\text{DES}(\overline{k}, \overline{m})}
$$

<p style="page-break-after:always;"></p>

UW user id: `g66xu`

# Problem 3
## a)
The decryption is as follows:

$$
D\vert_\text{4DES}((k_1, k_2, k_3, k_4), c) \mapsto D(k_1, D(k_2, D(k_3, D(k_4, c))))
$$

This decryption is obtained by successively removing the outermost layer of encryption using the appropriate key:

$$
\begin{aligned}
c &= E(k_4, E(k_3, E(k_2, E(k_1, m)))) \\
D(k_4, c) &= E(k_3, E(k_2, E(k_1, m))) \\
D(k_3, D(k_4, c)) &= E(k_2, E(k_1, m)) \\
D(k_2, D(k_3, D(k_4, c))) &= E(k_1, m) \\
D(k_1, D(k_2, D(k_3, D(k_4, c)))) &= m \\
\end{aligned}
$$

## b)
In a brute-force attack, there are $2^{56 \cdot 4} = 2^{224}$ possible key values to search through. For each 4DES key, the 4DES encryption involves 4 DES operations. Therefore, **the total time complexity of the attack is $2^{224} \cdot 4 = 2^{226}$ DES operations**. The space complexity is a constant since we only store one key at a time

## c)
We will describe a "meet at the exact middle" attack, though meeting at the quarter point or the three quarter point would also follow similar argument.

First we need to iterate through all possible values for the first two DES key. For each 2DES key value, we compute the first two rounds of encryption, and store the "half-ciphertext" alongside the corresponding "2DES key" in a sorted list. There are $2^{112}$ 2DES keys to search through, and each encryption takes 2 DES operations, so the time complexity is $2^{113}$ DES operations. Storing $2^{112}$ 2DES key (each 2DES key takes 112 bits) and half-ciphertext (each ciphertext is 64 bits) will take $176 \cdot 2^{112}$ bits fo storage, which is approximately $1.1 \cdot 10^{20}$ petabytes (here I count a petabyte as $2^{50}$ bytes).

In the second half of the attack, we iterate through all possible values of the other two keys and run decryption on the ciphertext for each of such key. There are $2^{112}$ keys to run decryption with, and each decryption costs 2 DES operations, so the second half of the attack takes $2^{113}$ DES operations.

In total, **the time complexity is $2^{113} + 2^{113} = 2^{114}$ DES operations, and it takes $176 \cdot 2^{112} \text{bits} \approx 1.1 \cdot 10^{20} \text{PB}$ space**.

## d)
Let $K_{12} = (k_1, k_2), K_{34} = (k_3, k_4)$. In the MITM attack described in part (c), we iterate through all possible $2^{112}$ values for $K_{12}$ and store all results.

When trading time for space, we can fix the first $s$ bits of $K_{12}$ and only store the remaining $s^{112 - s}$ pairs of $K_{12}$ and half-ciphertexts. For each of $2^s$ possible value for the fixed bits, we perform $2^{112 - s}$ 2DES encryptions, store $(112 + 64) \cdot 2^{112 - s}$ bits of half-ciphertext-half-key pairs, then perform $2^{112}$ decryptions during the second half of the MITM attack.

**The time complexity is $2^s \cdot (2^{112 -s} + s^{112}) = 2^{112} + 2^{112 + s} \approx 2^{112 + s}$ 2DES operations (or $2^{113 + s}$ DES operations). The space requirement is $176 \cdot 2^{112 - s}$ bits of storage.**

## e)
The key distinction between the two worlds is that in world B, there is a non-zero chance that the randomly sampled messages will collide, but in world A this chance is exactly 0 since all messages are guaranteed to be distinct. **An adversary can distinguish the two worlds by checking whether there are collisions in the ciphertexts: if there is collision, adversary reports world B, else it reports world A.**

According to an approximation on [Wikipedia](https://en.wikipedia.org/wiki/Birthday_problem#Approximations), the probability of having collision among $n$ people and $d$ possible days can be approximated using:

$$
P(\text{collision}) = 1 - e^{-\frac{n^2}{2d}}
$$

Since the block size of DES (and thus 4DES) is 64-bits, we know $d = 2^{64}$. Setting $P = 0.75$, we can solve that the minimum $n$ is:

$$
n \geq 2 \cdot 2^{64} \ln{4} \approx 2^{65.471}
$$

In other words, **with approximately $2^{65.471}$ samples, there is a 75% probability world B will produce a collision**, and the adversary will be able to detect which world it is in. Each sample is obtained by running 4 DES operations, so **getting this many samples takes $2^{67.471}$ DES operations**.

<p style="page-break-after:always;"></p>

UW user id: `g66xu`

# Problem 4

## CBC
An adversary who knows the encryption key but not the IV can mount **ciphertext-only attack** on ciphertexts produced by CBC mode of operation (with a catch!). This is because all except for the first ciphertext blocks can be encrypted using only the key and ciphertext block before, which the adversary has access to.

However, the first block is safe. This can be argued by considering the special case where we have CBC but only one ciphertext block:

$$
c[0] = E(k, m[0] \oplus \text{IV})
$$

Here the plaintext is obscured by an unknown IV that effectively serves as a one-time pad (assuming that IV is changed between messages and is unpredictable), which we know to have perfect secrecy. Therefore, the **first ciphertext block will have perfect secrecy**.

## CFB
Similar to CBC, the decryption circuit of CFB only relies on the encryption key and the previous ciphertext block. Therefore, an adversary can mount **ciphertext-only attack** on all except for the first ciphertext block.

Also similar to CBC, **the first ciphertext block will have perfect secrecy** (as long as IV is not reused) because $E(k, \text{IV})$ effectively serves as a one-time pad:

$$
c[0] = m[0] \oplus E(k, \text{IV})
$$

## OFB, CTR
In both CTR and OFB mode, the role of the encryption key and the IV is reversible (maybe except for when key space is larger than the message space). Again assuming that IV is not reused and is unpredictable, even if the adversary can decrypt arbitrary ciphertexts, those ciphertexts are encrypted under unknown and distinct IVs, so the adversary will not be able to deduce additional information.

In other words, when key is exposed and IV kept secret, **OFB and CTR mode still maintains semantic security**


<p style="page-break-after:always;"></p>

UW user id: `g66xu`

# Problem 5

## a)
Let $x = x[0] \Vert x[1] \in \mathbb{Z}_2^{2m}$ where $x[0], x[1] \in \mathbb{Z}^m$.

**$H$ does not have collision resistance** because the bitwise $\odot$ is commutative. We can first produce an $x$ where $x[0] \neq x[1]$, then we can produce a second $x^\prime = x[1] \Vert x[0] \neq x$ and they will have the same hash beacuse $x[0] \odot x[1] = x[1] \odot x[0]$

$$
H(x^\prime) = H(x[1] \Vert x[0]) = f(x[1] \odot x[0]) = f(x[0] \odot x[1]) = H(x)
$$

**$H$ does not have 2nd pre-image resistance** because negating both operands of the bitwise $\odot$ operator will not change the evaluation (two bits are equal if and only if their negation are equal). So given some $x = x[0] \Vert x[1]$, we know that $\overline{x} = \overline{x[0]} \Vert \overline{x[1]}$ will have the same hash (here the overline denotes bitwise negation):

$$
H(\overline{x}) = f(\overline{x[0]} \odot \overline{x[1]}) = f(x[0] \odot x[1]) = H(x)
$$

**$H$ has pre-image resistance**. We will prove by showing that if we break pre-image resistance for $H$, we can break pre-image for $f$.

Let $h$ be some arbitrary hash output of $H$. Assuming we can break pre-image resistance of $H$, we can find $x = x[0] \Vert x[1]$ such that $H(x) = h$. However, by the definition of $H$ we know that $H(x) = f(x[0] \odot x[1]) = h$, meaning that we have found some value $x^\prime = x[0] \odot x[1]$ such that $f(x^\prime) = h$. Thus we have found a pre-image of some arbitrary hash $h$ out of $f$.

## b)
When the input space for $H$ is 256-bit string, we know the output space to be 128-bit string. Disregarding all other ways of encountering a collision and relying on brute-force alone, we expect it to take $\sqrt{2^{128}} \approx 2^{64}$ uniformly random samples to produce a collision based on a generic birthday attack

<p style="page-break-after:always;"></p>

UW user id: `g66xu`

# Problem 6

## a)
**When the IV is fixed, an adversary can win the OW-CPA game by recovering the key stream using a single chosen plaintext/ciphertext pair**

Recall that in OFB mode, a key stream is computed by repeatedly encrypting the IV under the same encryption key. Let $c^*$ be the challenge ciphertext, $m^*$ be the corresponding message, $n$ be the length of the plaintext in "number of blocks", and $\text{IV}$ denote the fixed IV, then by the definition of OFB, we have the following:

$$
\begin{aligned}
c^* &= IV \\
&\Vert m^*[0] \oplus E(k, \text{IV}) \\
&\Vert m^*[1] \oplus E^2(k, \text{IV}) \\
&\ldots \\
&\Vert m^*[n-1] \oplus E^{n+1}(k, \text{IV}) \\
\end{aligned}
$$

For the adversary to win the OW-CPA game, the chosen plaintext $m_0$ will be some arbitrary bit string with (at least) $n$ blocks:

$$
m_0 = m_0[0] \Vert m_0[1] \Vert \ldots \Vert m_0[n-1]
$$

Then the output of the encryption oracle $c_0$ will be

$$
\begin{aligned}
c_0 &= IV \\
&\Vert m_0[0] \oplus E(k, \text{IV}) \\
&\Vert m_0[1] \oplus E^2(k, \text{IV}) \\
&\ldots \\
&\Vert m_0[n-1] \oplus E^n(k, \text{IV}) \\
\end{aligned}
$$

From here, the adversary can recover the key stream: $\text{key stream} = c_0 \oplus m_0$, and since the encryption oracle both the same encryption key and the same IV, the key stream is exactly what's used to encrypt the challenge plaintext, so the adversary can apply this key stream to the challenge ciphertext and recover the plaintext

## b)
**When each IV is guaranteed to be unique, the OFB mode is OW-CPA secure**. We will prove the OW-CPA security by showing that if there exists probabilistic polynomial-time (PPT) adversary who can win the OW-CPA game, then there exists a second PPT adversary who can distinguish the underlying block cipher from truly random permutation.

First, the OW-CPA adversary outputs the "challenge IV" and receives the challenge ciphertext. For the OW-CPA adversary to win the game, it can recover the message, and thus the key stream. Note that the first block of key stream is the encryption of the challenge IV: $E(k, IV^*)$.

Then, the block cipher adversary sends the challenge IV to the block cipher challenger and receives $h$, which has a 50% chance of being the output of the block cipher under the same key and 50% chance of being a truly random permutation. However, since the block cipher adversary has access to $E(k, IV^*)$ and the block cipher itself is necessarily deterministic, the block cipher adversary can check whether $h = E(k, IV^*)$. Since half of the time $h$ equals $E(k, IV^*)$, the advantage of the block cipher adversary is half the advantage of the OW-CPA adversary.

Therefore, if the OW-CPA adversary has non-negligible advantage, then the block cipher adversary will also have non-negligible advantage.